In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire.utils import decode

In [3]:
#Variables
website = 'https://jardinzoo.cl/roedores-167'
fileName = 'rodentsProducts'
folder = 'jardinzoo'

In [4]:
chrome_options = Options()
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
chrome_options.add_argument('--auto-open-devtools-for-tabs')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable_encoding')

driver = webdriver.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Network.enable", {})

driver.get(website)

wait = WebDriverWait(driver, 30)

In [5]:
#Products
products = []

In [6]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import random

def scan_products():
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article')))
    items = driver.find_elements(By.CSS_SELECTOR, 'article')
    
    for item in items:
        image_url = item.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
        product_url = item.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        title = item.find_element(By.CSS_SELECTOR, '.product-title').text
        price = item.find_element(By.CSS_SELECTOR, '.price').text
        products.append({
            'image_url': image_url,
            'product_url': product_url,
            'title': title,
            'price': price,
        })
    
def go_to_next_page():
    try:
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.next')))
        next_button.click()
        wait.until(EC.staleness_of(next_button))
        return True
    except NoSuchElementException:
        print("Botón 'next' no encontrado, final de la paginación.")
        return False
    except StaleElementReferenceException:
        print("El botón 'next' ya no es válido en el DOM.")
        return False
        
flag = True
while(flag):
    try:
        scan_products()
        time.sleep(random.uniform(3, 8))
        flag = go_to_next_page()
    except Exception as e:
        print(f"Error inesperado: {e}")
        flag = False
                

Error inesperado: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF620C0B095+29557]
	(No symbol) [0x00007FF620B7FA50]
	(No symbol) [0x00007FF620A3B56A]
	(No symbol) [0x00007FF620A8F695]
	(No symbol) [0x00007FF620A8F8EC]
	(No symbol) [0x00007FF620ADB777]
	(No symbol) [0x00007FF620AB71CF]
	(No symbol) [0x00007FF620AD851C]
	(No symbol) [0x00007FF620AB6F33]
	(No symbol) [0x00007FF620A8116F]
	(No symbol) [0x00007FF620A822D1]
	GetHandleVerifier [0x00007FF620F3C96D+3378253]
	GetHandleVerifier [0x00007FF620F88497+3688311]
	GetHandleVerifier [0x00007FF620F7D1CB+3642539]
	GetHandleVerifier [0x00007FF620CCA6B6+813462]
	(No symbol) [0x00007FF620B8AB5F]
	(No symbol) [0x00007FF620B86B74]
	(No symbol) [0x00007FF620B86D10]
	(No symbol) [0x00007FF620B75C1F]
	BaseThreadInitThunk [0x00007FF9D0D2257D+29]
	RtlUserThreadStart [0x00007FF9D2D2AF28+40]



In [7]:
print(products)
print(len(products))

[{'image_url': 'https://jardinzoo.cl/1891-home_default/alimento-para-mini-hamster-tropifit-150g.jpg', 'product_url': 'https://jardinzoo.cl/hamster/1686-alimento-para-mini-hamster-tropifit-150g-5900469532126.html', 'title': 'ALIMENTO PARA MINI HAMSTER TROPIFIT 150G', 'price': '$ 3.200'}, {'image_url': 'https://jardinzoo.cl/7723-home_default/arnes-mini-mascota-m.jpg', 'product_url': 'https://jardinzoo.cl/inicio/4768-arnes-mini-mascota-m-709081031292.html', 'title': 'ARNES MINI MASCOTA M', 'price': '$ 3.500'}, {'image_url': 'https://jardinzoo.cl/8734-home_default/brit-animals-guinea-pig-15kg.jpg', 'product_url': 'https://jardinzoo.cl/cuye-conejillo-de-indias/5196-brit-animals-guinea-pig-15kg-8595602504787.html', 'title': 'BRIT ANIMALS GUINEA PIG 1,5KG', 'price': '$ 20.950'}, {'image_url': 'https://jardinzoo.cl/9709-home_default/brit-animals-guinea-pig-300-grs.jpg', 'product_url': 'https://jardinzoo.cl/cuye-conejillo-de-indias/5538-brit-animals-guinea-pig-300-grs-8595602504794.html', 'titl

In [8]:
def go_to_product(product):
    wait = WebDriverWait(driver, 30)
    driver.get(product['product_url'])
    try:
        product_reference = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product['ean'] = product_reference.find_element(By.CSS_SELECTOR, 'span').text
    except NoSuchElementException:
        print(f"El elemento {product['title']} '.product-reference' no se encuentra en esta página")
        product['ean'] = ''

    try:
        product['brand'] = driver.find_element(By.CSS_SELECTOR, '.product-manufacturer').find_element(By.CSS_SELECTOR, 'a').text
    except NoSuchElementException:
        print(f"El elemento {product['title']} '.product-manufacturer' no se encuentra en esta página")
        product['brand'] = ''
        
    product['price'] = driver.find_element(By.CSS_SELECTOR, '.current-price').find_element(By.CSS_SELECTOR, 'span').get_attribute('content')
    product['currency'] = 'CLP'

for product in products:
    try:
        go_to_product(product)
    except Exception as e:
        print(f"Error inesperado producto {product['title']}: {e}")
    

El elemento ARNES MINI MASCOTA M '.product-manufacturer' no se encuentra en esta página
El elemento HENO HENOLET ALFALFA 100% NATURAL '.product-manufacturer' no se encuentra en esta página
El elemento JAULA CONEJERA R1 '.product-manufacturer' no se encuentra en esta página
El elemento JAULA CONEJERA R2 '.product-manufacturer' no se encuentra en esta página
El elemento LIVING WORLD EXTRUSION CONEJO 1,4 KG '.product-manufacturer' no se encuentra en esta página
El elemento LW BOTELLA XS 50 ML '.product-manufacturer' no se encuentra en esta página
El elemento OXBOW HENO DE AVENA 425GR '.product-manufacturer' no se encuentra en esta página
El elemento OXBOW HENO DE HIERBA DE HUERTO 425 GRS '.product-manufacturer' no se encuentra en esta página
El elemento OXBOW HENO WESTERN TIMOTHY 425G '.product-manufacturer' no se encuentra en esta página
El elemento PELLET ECOLOGICO 4KG '.product-manufacturer' no se encuentra en esta página
El elemento SUSTRATO MY PET LITTER 10 L '.product-manufacturer' 

In [9]:
print(products)

[{'image_url': 'https://jardinzoo.cl/1891-home_default/alimento-para-mini-hamster-tropifit-150g.jpg', 'product_url': 'https://jardinzoo.cl/hamster/1686-alimento-para-mini-hamster-tropifit-150g-5900469532126.html', 'title': 'ALIMENTO PARA MINI HAMSTER TROPIFIT 150G', 'price': '3200', 'ean': '5900469532126', 'brand': 'TROPIFIT', 'currency': 'CLP'}, {'image_url': 'https://jardinzoo.cl/7723-home_default/arnes-mini-mascota-m.jpg', 'product_url': 'https://jardinzoo.cl/inicio/4768-arnes-mini-mascota-m-709081031292.html', 'title': 'ARNES MINI MASCOTA M', 'price': '3500', 'ean': '709081031292OU', 'brand': '', 'currency': 'CLP'}, {'image_url': 'https://jardinzoo.cl/8734-home_default/brit-animals-guinea-pig-15kg.jpg', 'product_url': 'https://jardinzoo.cl/cuye-conejillo-de-indias/5196-brit-animals-guinea-pig-15kg-8595602504787.html', 'title': 'BRIT ANIMALS GUINEA PIG 1,5KG', 'price': '20950', 'ean': '8595602504787', 'brand': 'BRIT', 'currency': 'CLP'}, {'image_url': 'https://jardinzoo.cl/9709-home

In [10]:
with open(f"{folder}/{fileName}.json", 'w', encoding='utf-8') as json_file:
    json.dump(products, json_file, ensure_ascii=False, indent=4)

print("Archivo JSON generado correctamente.")

Archivo JSON generado correctamente.


In [11]:
driver.quit()